# Libraries

In [1]:
# General Libraries
import json
import numpy as np
import pandas as pd
import os
import ast
import inspect
from collections.abc import Iterable

# Base Classes & Estimators
from sklearn.base import BaseEstimator, TransformerMixin

# Pipeline & Model Construction
from imblearn.pipeline import Pipeline as imPipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

# Preprocessing & Transformation
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.neighbors import NeighborhoodComponentsAnalysis

# Feature Selection
from sklearn.feature_selection import RFE, SelectKBest

# Handling Imbalance
from imblearn.over_sampling import SMOTE

# Models
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Model Tuning & Cross-validation
from sklearn.model_selection import GridSearchCV, cross_validate, StratifiedKFold, train_test_split

# Model Evaluation & Scoring
from sklearn.metrics import classification_report, confusion_matrix, make_scorer


# Custom Classes

In [2]:
class OutlierClipper(BaseEstimator, TransformerMixin):
    def __init__(self, lower_percentile=0.005, upper_percentile=0.995, use_iqr=False):
        """
        Initialize the OutlierClipper with options for percentile clipping or IQR-based clipping.

        Parameters:
        - lower_percentile: float, lower bound percentile for clipping (if percentiles are used)
        - upper_percentile: float, upper bound percentile for clipping (if percentiles are used)
        - use_iqr: bool, whether to use IQR method for determining bounds
        """
        self.lower_percentile = lower_percentile
        self.upper_percentile = upper_percentile
        self.use_iqr = use_iqr
    
    def fit(self, X, y=None):
        """
        Fit the clipping bounds based on the training dataset using the specified method (percentiles or IQR).

        Parameters:
        - X: numpy.ndarray or pandas.DataFrame, the dataset used for fitting
        - y: ignored, not used for fitting

        Returns:
        - self: fitted instance of the class
        """
        # Convert to DataFrame if input is numpy array
        X = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X
        
        # For each column in X, calculate the bounds using the specified method
        self.bounds_ = {}
        for column in X.columns:
            if self.use_iqr:
                q1 = X[column].quantile(0.25)  # 1st quartile
                q3 = X[column].quantile(0.75)  # 3rd quartile
                iqr = q3 - q1  # Interquartile range
                lower_bound = q1 - 1.5 * iqr
                upper_bound = q3 + 1.5 * iqr
            else:
                lower_bound = X[column].quantile(self.lower_percentile)
                upper_bound = X[column].quantile(self.upper_percentile)

            self.bounds_[column] = (lower_bound, upper_bound)

        return self

    def transform(self, X):
        """
        Apply clipping to the dataset based on the fitted bounds.

        Parameters:
        - X: numpy.ndarray or pandas.DataFrame, the dataset to transform

        Returns:
        - X: pandas.DataFrame, the transformed dataset with clipped values
        """
        # Convert to DataFrame if input is numpy array
        X = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X

        # Apply clipping for each column
        for column, (lower_bound, upper_bound) in self.bounds_.items():
            X[column] = X[column].clip(lower=lower_bound, upper=upper_bound)

        return X

    def set_output(self, transform="default"):
        """
        Enable compatibility with scikit-learn's `set_output` functionality.

        Parameters:
        - transform: str, the output format ("default" or "pandas").
        """
        self.output_format = transform
        return self


In [3]:
class FrequencyEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # Compute the frequency counts for each column in the DataFrame
        self.freq_map = X.apply(pd.Series.value_counts)
        return self

    def transform(self, X):
        # Apply the frequency counts to transform the data
        return X.apply(lambda col: col.map(self.freq_map[col.name]))

# Data Ingestion

In [4]:
# Data ingestion with dtype application directly
with open('../data/dtypes.json', 'r') as file:
    dtypes = json.load(file)
train = pd.read_csv('../data/preproc_train.csv', index_col=0, dtype=dtypes, low_memory=True)
test = pd.read_csv('../data/preproc_test.csv', index_col=0, dtype=dtypes, low_memory=True)

# Handle NaN and target variable creation
train = train.fillna(np.nan)
X = train.drop(columns=['claim_injury_type'])
y = train['claim_injury_type'].map(lambda x: int(x[0]) - 1).astype(int)

# Extract rows where y is in [6, 7, 8], then sample remaining rows to match size
rows_with_78 = y.isin([6, 7])
X_with_78, y_with_78 = X[rows_with_78], y[rows_with_78]

# Sample from the rest of the data
X_remaining, y_remaining = X[~rows_with_78], y[~rows_with_78]
remaining_sample_size = 5000 - len(X_with_78)
X_sampled, y_sampled = X_remaining.sample(remaining_sample_size, random_state=42), y_remaining.loc[X_remaining.sample(remaining_sample_size, random_state=42).index]

# Concatenate the data
X, y = pd.concat([X_with_78, X_sampled]), pd.concat([y_with_78, y_sampled])

In [5]:
metric_features = [
    'age_at_injury', 'ime_4_count', 'average_weekly_wage', 'birth_year', 'number_of_dependents', 'dd_asb_c2', 'dd_asb_c3', 'dd_c2_c3',
    'first_hearing_date_day', 'first_hearing_date_month', 'first_hearing_date_year', 'c_2_date_day', 'c_2_date_month', 'c_2_date_year',
    'c_3_date_day', 'c_3_date_month', 'c_3_date_year', 'assembly_date_day', 'assembly_date_month', 'assembly_date_year',
    'accident_date_day', 'accident_date_month', 'accident_date_year', 'avg_word_emb_dim_0', 'avg_word_emb_dim_1', 'avg_word_emb_dim_2',
    'avg_word_emb_dim_3', 'avg_word_emb_dim_4', 'avg_word_emb_dim_5', 'avg_word_emb_dim_6', 'avg_word_emb_dim_7', 'avg_word_emb_dim_8',
    'avg_word_emb_dim_9', 'var_word_emb_dim_0', 'var_word_emb_dim_1', 'var_word_emb_dim_2', 'var_word_emb_dim_3', 'var_word_emb_dim_4',
    'var_word_emb_dim_5', 'var_word_emb_dim_6', 'var_word_emb_dim_7', 'var_word_emb_dim_8', 'var_word_emb_dim_9', 'euclidean_norm'
]

binary_features = [
    'age_at_injury_zero', 'is_unionized', 'alternative_dispute_resolution', 'attorney_representative', 'covid_19_indicator', 'do_1', 'do_10',
    'do_11', 'do_12', 'do_13', 'do_14', 'do_15', 'do_16', 'do_2', 'do_3', 'do_4', 'do_5', 'do_6', 'do_7', 'do_8', 'do_9', 'missing_accident_date',
    'missing_age_at_injury', 'missing_average_weekly_wage', 'missing_birth_year', 'missing_c_2_date', 'missing_c_3_date', 'missing_first_hearing_date',
    'missing_gender', 'missing_ime_4_count', 'missing_industry_code', 'missing_industry_code_description', 'missing_wcio_cause_of_injury_code',
    'missing_wcio_cause_of_injury_description', 'missing_wcio_nature_of_injury_code', 'missing_wcio_nature_of_injury_description',
    'missing_wcio_part_of_body_code', 'missing_wcio_part_of_body_description', 'missing_zip_code'
]

hot_columns = ["carrier_type", "part_of_body_group", "cause_of_injury_group", "medical_fee_region"]
frequency_columns = ["industry_code"]

In [6]:
# Due to the extremely high computational demands of Support-Vector Machines in general, with this algorithm we decide to start from a smaller set of columns.

custom_column_selection = [
    'attorney_representative', 'average_weekly_wage', 'ime_4_count', 'gender', 'age_at_injury',
    'do_14', 'do_3', 'do_12', 'do_6', 'do_2', 'do_10', 'do_13', 'missing_c_3_date', 
    'missing_first_hearing_date', 'missing_ime_4_count',
    'avg_word_emb_dim_0', 'avg_word_emb_dim_1', 'avg_word_emb_dim_2', 'avg_word_emb_dim_3', 
    'avg_word_emb_dim_4', 'avg_word_emb_dim_5', 'avg_word_emb_dim_6', 'avg_word_emb_dim_7', 
    'avg_word_emb_dim_8', 'avg_word_emb_dim_9',
    'var_word_emb_dim_0', 'var_word_emb_dim_1', 'var_word_emb_dim_2', 'var_word_emb_dim_3', 
    'var_word_emb_dim_4', 'var_word_emb_dim_5', 'var_word_emb_dim_6', 'var_word_emb_dim_7', 
    'var_word_emb_dim_8', 'var_word_emb_dim_9',
    'euclidean_norm'
]

X = X[custom_column_selection]


# Pipeline

In [12]:
pipeline = imPipeline(steps=[
    ('outlier_clipper', OutlierClipper()),  # Handle outliers
    ('scaler', StandardScaler()),  # Standardize features
    ('knnimputer', KNNImputer(n_neighbors=5)),  # Impute missing values using KNN
    ('pca', PCA(n_components=0.9)),  # Reduce dimensionality (90% variance)
    ('select', SelectKBest()),  # Neighborhood Components Analysis
    ('smote', SMOTE(sampling_strategy='auto', k_neighbors=5)),  # Handle class imbalance with SMOTE
    ('svm', SVC( kernel='rbf', class_weight='balanced'))  # SVM with RBF kernel and balanced class weights
])

In [13]:
param_grid = {
    'select__k' : [10, 15, 20],
    # SVM hyperparameters to increase penalization and prevent overfitting
    'svm__C': [0.5, .75, 1]  # Higher C values lead to stronger regularization (penalization)
}

# Fit

In [14]:
outer_cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
inner_cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=inner_cv,
    n_jobs=1,
    refit=True,
    verbose=2
)

# Perform cross-validation using GridSearchCV with outer cross-validation
cv_results = cross_validate(
    estimator=grid_search,
    X=X,
    y=y,
    cv=outer_cv,
    return_train_score=True,
    return_estimator=True,
    scoring="f1_macro",
    n_jobs=1,
    verbose=2
)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV] END ...........................select__k=10, svm__C=0.5; total time=   0.9s
[CV] END ...........................select__k=10, svm__C=0.5; total time=   0.9s
[CV] END ..........................select__k=10, svm__C=0.75; total time=   0.8s
[CV] END ..........................select__k=10, svm__C=0.75; total time=   0.9s
[CV] END .............................select__k=10, svm__C=1; total time=   0.8s
[CV] END .............................select__k=10, svm__C=1; total time=   0.8s
[CV] END ...........................select__k=15, svm__C=0.5; total time=   0.9s
[CV] END ...........................select__k=15, svm__C=0.5; total time=   0.9s
[CV] END ..........................select__k=15, svm__C=0.75; total time=   0.8s
[CV] END ..........................select__k=15, svm__C=0.75; total time=   0.8s
[CV] END .............................select__k=15, svm__C=1; total time=   0.8s
[CV] END .............................select__k=1

/home/nottoriousgg/anaconda3/envs/MachineLearning/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:776: UserWarning: k=20 is greater than n_features=18. All the features will be returned.
  warnings.warn(


[CV] END ...........................select__k=20, svm__C=0.5; total time=   0.9s


/home/nottoriousgg/anaconda3/envs/MachineLearning/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:776: UserWarning: k=20 is greater than n_features=18. All the features will be returned.
  warnings.warn(


[CV] END ...........................select__k=20, svm__C=0.5; total time=   1.0s


...

[CV] END .............................select__k=20, svm__C=1; total time=   0.8s


/home/nottoriousgg/anaconda3/envs/MachineLearning/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:776: UserWarning: k=20 is greater than n_features=18. All the features will be returned.
  warnings.warn(


[CV] END .............................select__k=20, svm__C=1; total time=   0.8s


/home/nottoriousgg/anaconda3/envs/MachineLearning/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:776: UserWarning: k=20 is greater than n_features=18. All the features will be returned.
  warnings.warn(


[CV] END .................................................... total time=  20.3s


# Results

In [15]:
# Convert cv_results to a pandas DataFrame
cv_results_df = pd.DataFrame(cv_results)

# Specify the file path (in the current working directory)
file_path = '../results/cv_results.csv'

# Append the DataFrame to the CSV file (if it exists) or create a new one
cv_results_df.to_csv(file_path, mode='a', header=not pd.io.common.file_exists(file_path), index=False)

print(f"Cross-validation results saved to {file_path}")

Cross-validation results saved to ../results/cv_results.csv


In [18]:
cv_results_df.iloc[1]['estimator'].best_estimator_

Pipeline(steps=[('outlier_clipper', OutlierClipper()),
                ('scaler', StandardScaler()), ('knnimputer', KNNImputer()),
                ('pca', PCA(n_components=0.9)), ('select', SelectKBest(k=20)),
                ('smote', SMOTE()),
                ('svm', SVC(C=0.5, class_weight='balanced'))])

# Predict

In [ ]:
# Extract the best estimator from cross-validation results
best_pipeline = cv_results_df.iloc[1]['estimator'].best_estimator_

# Fit the best pipeline on the entire dataset
best_pipeline.fit(train[custom_column_selection].copy()), y)

# Get predictions from the test set
predictions = best_pipeline.predict(test[custom_column_selection].copy())

# Decode the predictions using the inverse mapping
decoded_predictions = pd.Series(predictions).map(inverse_mapping)

# Convert the index back to a column
predictions_with_index = decoded_predictions.reset_index()

# Export predictions with index to CSV
predictions_with_index.to_csv('predictions_with_index.csv', index=False)
